In [ ]:
# Naukri.com web scraping
# Name: Shweta Ghadshi
# BE - CMPN B
# Roll no: 21102B0017

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

def send_job_name(job_roles, driver):
    driver.get('https://www.naukri.com/')
    df = pd.DataFrame({'Skill': [''], 'Link': [''], 'Name': [''], 'Company_Name': [''], 'Experience': [''], 'Location': [''], 'Job_Description': [''], 'Required_skills': ['']})
    
    for job_role in job_roles:
        print(f"Scraping job postings for: {job_role}")
        if job_role == job_roles[0]:
            input_search = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/div[7]/div/div/div[1]/div/div/div/div[1]/div/input'))
            )
            input_search.send_keys(job_role)
            search_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[7]/div/div/div[6]'))
            )
            search_button.click()
        else:
            logo_button = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div[2]/a/img'))
            )
            logo_button.click() 
            input_search = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/div[6]/div/div/div[1]/div/div/div/div[1]/div/input'))
            )
            input_search.send_keys(job_role)
            search_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[6]/div/div/div[6]'))
            )
            search_button.click()

        n = 2

        while n > 0:
            WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'cust-job-tuple'))
            )
            soup = BeautifulSoup(driver.page_source, 'lxml')
            posting = soup.find_all('div', class_ = 'cust-job-tuple')
            for post in posting:
                link = post.find('a', class_='title').get('href')
                name = post.find('a', class_='title').text
                row2_div = post.find('div', class_='row2')
                company_name_tag = row2_div.find('span', class_='comp-dtls-wrap') if row2_div else None
                row2_div_a = company_name_tag.find('a', 'comp-name') if company_name_tag else None
                company_name = row2_div_a.text if row2_div_a else 'No company name' 
                experience_tag = post.find('span', class_='expwdth')
                experience = experience_tag.text if experience_tag else 'No experience'
                location_tag = post.find('span', class_='locWdth')
                location = location_tag.text if location_tag else 'No location'
                job_description_tag = post.find('span', class_='job-desc')
                job_description = job_description_tag.text if job_description_tag else 'No job description'
                required_skills = ', '.join(li_tag.text.strip() for li_tag in post.find('ul', class_='tags-gt').find_all('li', class_='tag-li')) if post.find('ul', class_='tags-gt') else 'No required skills'
                df = df.append({'Skill': job_role, 'Link': link, 'Name': name, 'Company_Name': company_name, 'Experience': experience, 'Location': location, 'Job_Description': job_description, 'Required_skills': required_skills}, ignore_index = True)
            try: 
                next_button = WebDriverWait(driver, 20).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="lastCompMark"]/a[2]'))
                )
                next_button.click()
                WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.CLASS_NAME, 'cust-job-tuple'))
                )
                time.sleep(2)
            except:
                print("Error Occured")
            n -= 1
    return df

In [3]:
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration, typically for Windows OS
# chrome_options.add_argument("--window-size=1920,1080")  # Set a default window size
# chrome_options.add_argument("--no-sandbox")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

job_roles = ['react', 'javascript']

all_jobs_df = send_job_name(job_roles, driver)

all_jobs_df.to_csv('job_postings.csv', index=False)

print('Done')
driver.quit()

Scraping job postings for: react
Scraping job postings for: javascript
Done
